In [4]:
import numpy as np
import cv2
cap = cv2.VideoCapture('intersection.mp4')
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

#Counter used to trigger new points to track
n=0


while(1):
    n +=1
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame_gray = fgmask
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    if n == 100:
        p0=cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        n = 0
cv2.destroyAllWindows()
cap.release()

200


KeyboardInterrupt: 

KeyboardInterrupt: 

In [5]:
import numpy as np
import cv2
cap = cv2.VideoCapture('intersection.mp4')
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
print(p0)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

#Counter used to trigger new points to track
n=0
while(1):
    print()
    n +=1
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame_gray = fgmask
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    #cv2.imshow('frame',img)
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    if n == 100:
        p0=cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        n = 0
cv2.destroyAllWindows()
cap.release()

[[[ 722.  315.]]

 [[ 555.  297.]]

 [[ 653.  263.]]

 [[ 839.  456.]]

 [[ 486.  310.]]

 [[ 816.  328.]]

 [[ 720.  414.]]

 [[ 962.  209.]]

 [[ 795.  421.]]

 [[ 829.  561.]]

 [[ 559.  495.]]

 [[ 240.  571.]]

 [[ 475.  307.]]

 [[ 467.  576.]]

 [[ 288.  279.]]

 [[ 657.  270.]]

 [[ 855.  334.]]

 [[ 986.  542.]]

 [[ 189.  414.]]

 [[ 545.  290.]]

 [[ 747.  607.]]

 [[ 957.  223.]]

 [[1021.  590.]]

 [[ 556.  378.]]

 [[ 410.  233.]]

 [[1000.  249.]]

 [[ 548.  299.]]

 [[ 894.  433.]]

 [[ 349.  346.]]

 [[ 305.  265.]]

 [[ 187.  422.]]

 [[1012.  571.]]

 [[ 370.  340.]]

 [[1110.  467.]]

 [[1114.  457.]]

 [[ 551.  436.]]

 [[ 373.  348.]]

 [[ 700.  331.]]

 [[ 493.  283.]]

 [[1041.  623.]]

 [[ 664.  270.]]

 [[ 226.  507.]]

 [[ 602.  273.]]

 [[1074.  178.]]

 [[ 930.  548.]]

 [[1056.  461.]]

 [[ 356.  350.]]

 [[ 950.  531.]]

 [[ 985.  503.]]

 [[ 555.  590.]]

 [[ 841.  267.]]

 [[ 515.  229.]]

 [[ 874.  433.]]

 [[ 715.  329.]]

 [[1015.  162.]]

 [[ 925.  

[[[ 722.2076    314.87817 ]]

 [[ 555.0922    296.8407  ]]

 [[ 653.2319    262.8354  ]]

 [[ 839.3648    455.78345 ]]

 [[ 486.16068   309.84622 ]]

 [[ 816.2278    327.91086 ]]

 [[ 720.19165   413.90668 ]]

 [[ 961.13715   207.29407 ]]

 [[ 783.59216   426.30423 ]]

 [[ 829.2665    560.92163 ]]

 [[ 560.2256    494.05682 ]]

 [[ 240.20668   570.671   ]]

 [[ 475.14594   306.8378  ]]

 [[ 444.5338    586.85693 ]]

 [[ 288.20053   278.78992 ]]

 [[ 657.2284    269.83456 ]]

 [[ 856.37573   333.34088 ]]

 [[ 986.1848    541.96124 ]]

 [[ 188.38504   416.7483  ]]

 [[ 545.1538    289.83652 ]]

 [[ 749.6118    605.74493 ]]

 [[ 954.2373    218.6279  ]]

 [[1021.3118    589.995   ]]

 [[ 556.19617   377.8707  ]]

 [[ 410.2592    232.8505  ]]

 [[ 999.0472    247.19565 ]]

 [[ 548.1212    298.8434  ]]

 [[ 894.2355    432.95627 ]]

 [[ 349.11972   345.85593 ]]

 [[ 305.18567   264.8321  ]]

 [[ 186.4847    424.79822 ]]

 [[1012.3143    570.93744 ]]

 [[ 370.1545    339.85422 ]]

 [[1110.28

[[[ 722.4141    314.73468 ]]

 [[ 555.2855    296.70026 ]]

 [[ 653.4016    262.7259  ]]

 [[ 839.6084    455.657   ]]

 [[ 486.29852   309.7255  ]]

 [[ 816.4101    327.7451  ]]

 [[ 720.3622    413.77118 ]]

 [[ 960.57947   206.25917 ]]

 [[ 776.5876    429.47928 ]]

 [[ 829.4102    560.76465 ]]

 [[ 560.95325   493.41748 ]]

 [[ 240.2918    570.65015 ]]

 [[ 475.32928   306.70428 ]]

 [[ 430.98206   593.39014 ]]

 [[ 288.3713    278.5974  ]]

 [[ 657.40753   269.7163  ]]

 [[ 857.28015   332.86896 ]]

 [[ 986.36115   541.82574 ]]

 [[ 188.17752   418.37497 ]]

 [[ 545.317     289.73584 ]]

 [[ 751.3035    605.02466 ]]

 [[ 952.70087   216.19539 ]]

 [[1021.4807    589.8474  ]]

 [[ 556.3416    377.741   ]]

 [[ 410.3633    232.69308 ]]

 [[ 998.56494   246.22461 ]]

 [[ 548.2988    298.70724 ]]

 [[ 894.4026    432.81467 ]]

 [[ 349.28583   345.7031  ]]

 [[ 305.37476   264.7502  ]]

 [[ 186.22337   426.4405  ]]

 [[1012.49554   570.8451  ]]

 [[ 370.32874   339.68503 ]]

 [[1110.50

[[[ 722.2944   314.70288]]

 [[ 555.20105  296.66962]]

 [[ 653.26685  262.66815]]

 [[ 839.61035  455.65207]]

 [[ 486.21414  309.6879 ]]

 [[ 816.3606   327.7319 ]]

 [[ 720.35614  413.76855]]

 [[ 959.84717  205.23418]]

 [[ 769.2054   432.81354]]

 [[ 829.37714  560.7201 ]]

 [[ 561.53107  492.7623 ]]

 [[ 240.22963  570.58704]]

 [[ 475.22842  306.6547 ]]

 [[ 417.10458  600.08295]]

 [[ 288.28738  278.59064]]

 [[ 657.2739   269.6862 ]]

 [[ 857.89856  332.57693]]

 [[ 986.36     541.80707]]

 [[ 187.55267  420.00183]]

 [[ 545.2523   289.7235 ]]

 [[ 752.8963   604.3093 ]]

 [[ 950.98285  213.69513]]

 [[1021.5037   589.8497 ]]

 [[ 556.30396  377.67142]]

 [[ 410.228    232.67072]]

 [[ 997.7914   245.24449]]

 [[ 548.1848   298.65186]]

 [[ 894.3961   432.80524]]

 [[ 349.21936  345.6599 ]]

 [[ 305.2254   264.6644 ]]

 [[ 185.63853  428.1086 ]]

 [[1012.48193  570.8197 ]]

 [[ 370.289    339.70273]]

 [[1110.5057   466.94254]]

 [[1114.4609   456.97824]]

 [[ 551.3091   435.7

[[[ 722.3467    314.62558 ]]

 [[ 555.3041    296.56293 ]]

 [[ 653.36346   262.589   ]]

 [[ 839.8191    455.56665 ]]

 [[ 486.26147   309.58176 ]]

 [[ 816.4026    327.6722  ]]

 [[ 720.41      413.69565 ]]

 [[ 959.3082    204.22931 ]]

 [[ 761.6603    436.26736 ]]

 [[ 829.3952    560.6577  ]]

 [[ 562.01526   492.3532  ]]

 [[ 240.049     570.4492  ]]

 [[ 475.30212   306.57037 ]]

 [[ 403.25262   606.7841  ]]

 [[ 288.32492   278.46115 ]]

 [[ 657.3481    269.58405 ]]

 [[ 858.66846   332.17352 ]]

 [[ 986.475     541.7428  ]]

 [[ 187.09923   421.72952 ]]

 [[ 545.3064    289.5751  ]]

 [[ 754.6988    603.417   ]]

 [[ 949.63434   211.44258 ]]

 [[1021.4999    589.80164 ]]

 [[ 556.35236   377.57855 ]]

 [[ 410.32742   232.56944 ]]

 [[ 997.19434   244.1177  ]]

 [[ 548.2627    298.5328  ]]

 [[ 894.541     432.75073 ]]

 [[ 349.24814   345.5311  ]]

 [[ 305.30154   264.5431  ]]

 [[ 185.1938    429.8353  ]]

 [[1012.5132    570.7504  ]]

 [[ 370.28156   339.54932 ]]

 [[1110.66

[[[ 722.2502    314.65326 ]]

 [[ 555.1574    296.59235 ]]

 [[ 653.25244   262.6037  ]]

 [[ 839.6914    455.59924 ]]

 [[ 486.10663   309.62485 ]]

 [[ 816.3021    327.70203 ]]

 [[ 720.23236   413.7332  ]]

 [[ 958.4533    203.34314 ]]

 [[ 753.88184   439.68613 ]]

 [[ 829.3078    560.6616  ]]

 [[ 562.60913   491.92822 ]]

 [[ 239.92578   570.5105  ]]

 [[ 475.1556    306.61142 ]]

 [[ 389.34695   613.7805  ]]

 [[ 288.1419    278.49268 ]]

 [[ 657.2061    269.61493 ]]

 [[ 859.27576   331.66873 ]]

 [[ 986.49133   541.81165 ]]

 [[ 186.4946    423.4529  ]]

 [[ 545.18616   289.61157 ]]

 [[ 756.577     602.5998  ]]

 [[ 948.17303   209.28728 ]]

 [[1021.5078    589.81714 ]]

 [[ 556.14166   377.6179  ]]

 [[ 410.235     232.57742 ]]

 [[ 996.39044   243.31558 ]]

 [[ 548.1306    298.58392 ]]

 [[ 894.35767   432.77896 ]]

 [[ 349.04053   345.56418 ]]

 [[ 305.15662   264.56644 ]]

 [[ 184.63048   431.57047 ]]

 [[1012.48474   570.7933  ]]

 [[ 370.08905   339.5881  ]]

 [[1110.55

[[[ 722.27356   314.61618 ]]

 [[ 555.13354   296.52872 ]]

 [[ 653.30536   262.56244 ]]

 [[ 839.81836   455.56064 ]]

 [[ 486.0907    309.5275  ]]

 [[ 816.32275   327.64597 ]]

 [[ 720.2576    413.6897  ]]

 [[ 957.52893   201.76083 ]]

 [[ 740.5822    445.91583 ]]

 [[ 829.22577   560.5457  ]]

 [[ 564.0495    491.0915  ]]

 [[ 239.67337   570.4029  ]]

 [[ 475.17517   306.52563 ]]

 [[ 365.70654   625.51575 ]]

 [[ 288.14664   278.4366  ]]

 [[ 657.27637   269.59097 ]]

 [[ 860.4845    331.18356 ]]

 [[ 986.5212    541.83813 ]]

 [[ 185.7489    426.25763 ]]

 [[ 545.2064    289.57654 ]]

 [[ 759.9934    601.0917  ]]

 [[ 946.39294   205.70865 ]]

 [[1021.4909    589.82043 ]]

 [[ 556.1195    377.55716 ]]

 [[ 410.2724    232.52486 ]]

 [[ 995.5834    241.78719 ]]

 [[ 548.1565    298.53223 ]]

 [[ 894.3672    432.77216 ]]

 [[ 349.0593    345.46548 ]]

 [[ 305.18005   264.49478 ]]

 [[ 183.80058   434.32156 ]]

 [[1012.403     570.91846 ]]

 [[ 370.04147   339.52652 ]]

 [[1110.58

[[[ 722.2177    314.54102 ]]

 [[ 555.07465   296.44916 ]]

 [[ 653.2656    262.49832 ]]

 [[ 839.76666   455.50403 ]]

 [[ 486.0226    309.41113 ]]

 [[ 816.2768    327.59033 ]]

 [[ 720.1332    413.64594 ]]

 [[ 956.59796   200.42868 ]]

 [[ 729.4408    450.86816 ]]

 [[ 829.2865    560.3726  ]]

 [[ 565.10236   490.39252 ]]

 [[ 239.51466   570.2444  ]]

 [[ 475.0772    306.40903 ]]

 [[ 346.78278   634.7017  ]]

 [[ 288.0653    278.2606  ]]

 [[ 657.23145   269.50366 ]]

 [[ 861.16943   330.63168 ]]

 [[ 986.42896   541.787   ]]

 [[ 185.04353   428.22156 ]]

 [[ 545.1263    289.4561  ]]

 [[ 763.1189    599.52625 ]]

 [[ 944.8593    202.88667 ]]

 [[1021.4165    589.7789  ]]

 [[ 555.9867    377.4823  ]]

 [[ 410.20764   232.37032 ]]

 [[ 994.60364   240.47527 ]]

 [[ 548.09644   298.43234 ]]

 [[ 894.2955    432.7484  ]]

 [[ 348.9029    345.32657 ]]

 [[ 305.11945   264.29538 ]]

 [[ 183.02779   436.3291  ]]

 [[1012.2563    570.78534 ]]

 [[ 369.93527   339.37152 ]]

 [[1110.55

[[[ 722.17163   314.50574 ]]

 [[ 555.0798    296.43082 ]]

 [[ 653.2197    262.46368 ]]

 [[ 839.68805   455.4524  ]]

 [[ 485.96173   309.37732 ]]

 [[ 816.1833    327.57205 ]]

 [[ 720.0228    413.61267 ]]

 [[ 955.72125   199.26123 ]]

 [[ 717.99414   455.99603 ]]

 [[ 829.2754    560.3212  ]]

 [[ 566.21375   489.7709  ]]

 [[ 239.32338   570.15436 ]]

 [[ 474.99908   306.3711  ]]

 [[ 327.44016   644.4297  ]]

 [[ 287.94345   278.22986 ]]

 [[ 657.1761    269.47702 ]]

 [[ 862.168     330.27292 ]]

 [[ 986.37286   541.7688  ]]

 [[ 184.3255    430.3467  ]]

 [[ 545.0527    289.41708 ]]

 [[ 766.39417   597.8991  ]]

 [[ 943.45966   200.3043  ]]

 [[1021.317     589.79486 ]]

 [[ 555.86597   377.45166 ]]

 [[ 410.1305    232.34718 ]]

 [[ 993.7815    239.35164 ]]

 [[ 548.0277    298.39023 ]]

 [[ 894.2135    432.7287  ]]

 [[ 348.7674    345.30283 ]]

 [[ 305.04498   264.29007 ]]

 [[ 182.34872   438.4525  ]]

 [[1012.17334   570.7945  ]]

 [[ 369.8132    339.35522 ]]

 [[1110.51

KeyboardInterrupt: 

In [6]:
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
color =(150,100,3)
mask = np.zeros_like(old_frame)
cap.release()
_number_of_frames = 0

def opticalFlowTracking(pointArray2Track):
    p0 = pointArray2Track
    ret,frame=cap.read()
    frame_grey =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray,frame_grey,p0,None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    # Drawing the points
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    cv2.imshow(img,"frame")
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    cv2.destroyAllWindows()
    cap.release()
